In [1]:
(*
 * Simple arithmetic language
 *)
type exp = 
  | Val of int
  | Plus of exp * exp
  | Div of exp * exp

type exp = Val of int | Plus of exp * exp | Div of exp * exp


In [2]:
let my_exp1 = Plus (Div (Val 4, Val 2), Val 6)

val my_exp1 : exp = Plus (Div (Val 4, Val 2), Val 6)


In [3]:
let rec eval (e:exp) : int = match e with
  | Val n -> n
  | Plus(e1,e2) ->
    let v1 = eval e1 in
    let v2 = eval e2 in
    v1+v2
  | Div (e1,e2) ->
    let v1 = eval e1 in
    let v2 = eval e2 in
    v1/v2

val eval : exp -> int = <fun>


In [4]:
eval my_exp1

- : int = 8


In [5]:
let my_exp2 = Plus (Div (Val 4, Val 0), Val 6)

val my_exp2 : exp = Plus (Div (Val 4, Val 0), Val 6)


In [6]:
eval my_exp2

error: runtime_error

In [7]:
let rec eval (e:exp) : int option = match e with
  | Val n -> Some n
  | Plus(e1,e2) ->
    let v1 = eval e1 in
    begin match v1 with
      | None -> None
      | Some n1 ->
        let v2 = eval e2 in
        begin match v2 with
          | None -> None
          | Some n2 -> Some (n1+n2)
        end
    end
  | Div (e1,e2) ->
    let v1 = eval e1 in
    begin match v1 with
      | None -> None
      | Some n1 ->
        let v2 = eval e2 in
        begin match v2 with
          | None -> None
          | Some 0 -> None
          | Some n2 -> Some (n1/n2)
        end
    end

val eval : exp -> int option = <fun>


In [8]:
eval my_exp2

- : int option = None


In [9]:
(*
 * For any 2 computations f and g:
 * 1. Do computation f.
 * 2. If result of f is None then return None.
 * 3. If the result of f is Some v, then execute (g v)
 *)
 let bind (v : int option) (g: int -> int option) : int option = 
   match v with
     | Some v1 -> g v1
     | None -> None
 
let return (v:int): int option = Some v 

val bind : int option -> (int -> int option) -> int option = <fun>


val return : int -> int option = <fun>


In [10]:
let rec eval (e:exp) : int option = match e with
  | Val n -> Some n
  | Plus(e1,e2) -> 
      bind (eval e1) 
           (fun v1 -> bind (eval e2) 
                           (fun v2 -> return (v1+v2))) 
  | Div (e1,e2) ->
      bind (eval e1) 
           (fun v1 -> bind (eval e2) 
                           (fun v2 -> if v2 = 0 then None
                                      else return (v1/v2)))

val eval : exp -> int option = <fun>


In [11]:
eval my_exp2

- : int option = None


In [12]:
let (>>=) = bind

val ( >>= ) : int option -> (int -> int option) -> int option = <fun>


In [13]:
let rec eval (e:exp) : int option = match e with
  | Val n -> Some n
  | Plus(e1,e2) -> 
      eval e1 >>= fun v1 -> 
      eval e2 >>= fun v2 -> 
      return (v1+v2)
  | Div (e1,e2) ->
      eval e1 >>= fun v1 -> 
      eval e2 >>= fun v2 ->
      if v2 = 0 then None else return (v1/v2)

val eval : exp -> int option = <fun>


In [14]:
let (let*) = bind

val ( let* ) : int option -> (int -> int option) -> int option = <fun>


In [15]:
let rec eval (e:exp) : int option = match e with
  | Val n -> Some n
  | Plus(e1,e2) -> 
      let* v1 = eval e1 in
      let* v2 = eval e2 in
      return (v1+v2)
  | Div (e1,e2) ->
      let* v1 = eval e1 in
      let* v2 = eval e2 in
      if v2 = 0 then None else return (v1/v2)

val eval : exp -> int option = <fun>


In [16]:
let rec eval_exn (e:exp) : int = match e with
  | Val n -> n
  | Plus(e1,e2) ->
    let v1 = eval_exn e1 in
    let v2 = eval_exn e2 in
    v1+v2
  | Div (e1,e2) ->
    let v1 = eval_exn e1 in
    let v2 = eval_exn e2 in
    v1/v2

val eval_exn : exp -> int = <fun>


In [17]:
type 'a t = 'a option
let return (v:'a) : 'a t = Some v
let bind (v:'a option) (f : 'a -> 'a option) : 'a option = 
  match v with
  | None -> None
  | Some v -> f v

type 'a t = 'a option


val return : 'a -> 'a t = <fun>


val bind : 'a option -> ('a -> 'a option) -> 'a option = <fun>


In [18]:
module type MONAD = sig
type 'a t
val return: 'a -> 'a t
val bind: 'a t -> ('a -> 'a t) -> 'a t
end

module type MONAD =
  sig
    type 'a t
    val return : 'a -> 'a t
    val bind : 'a t -> ('a -> 'a t) -> 'a t
  end


In [19]:
let list_return (v: 'a) : 'a list = [v]
(* val list_bind : 'a list -> ('a -> 'a list) -> 'a list *)
let list_bind (l: 'a list) (f: 'a -> 'a list) : 'a list = 
  List.concat (List.map f l)

val list_return : 'a -> 'a list = <fun>


val list_bind : 'a list -> ('a -> 'a list) -> 'a list = <fun>


### Monad Laws


1. Left Identity: `(return v) >>= f` = `f v` 
2. Right Identity: `f >>= return = f`
3. Associativity: `(m >>= f) >>= g = m >>= (fun x -> f x >= g)`

In [20]:
(return 2) >>= f
=
bind (return 2) f
=
match return 2 with
 | None -> None
 | Some 2 -> f 2
=
match Some 2 with
 | None -> None
 | Some 2 -> f 2
=
f 2

error: compile_error

```ocaml
(*
 * In OCaml
 *)
  let * v1 = c1 in
  let * v2 = c2 v1 in
  ...
  return vn
```


```haskell
(*
 * In Haskell
 *)
 do {
   v1 <- c1;
   v2 <- c2 v1;
   ...
   return vn
 }
 ```

In [21]:
(*
 * Mutations
 * New syntax:
 *  1. ref: create a reference
 *  2. ! : dereference a reference
 *  3. := : Update a reference.
 *)
let count = ref 0 (* int *count *)

val count : int ref = {contents = 0}


In [22]:
count := !count + 1

- : unit = ()


In [23]:
!count (* *ptr *)

- : int = 1


In [24]:
(*
 * cnt and count are aliases
 *)
let cnt = count

val cnt : int ref = {contents = 1}


In [25]:
cnt := !cnt + 10

- : unit = ()


In [26]:
!count

- : int = 11


In [27]:
let count = 1 in
let cnt = count in
let cnt = cnt + 1 in
count

File "[27]", line 3, characters 4-7:
3 | let cnt = cnt + 1 in
        ^^^
Warning 26 [unused-var]: unused variable cnt.


- : int = 1


```ocaml
let x = f () in
let y = f () in
x+y
```

```ocaml
let x = f () in
let y = x in
x+y
```

In [28]:
(*
 * let x = e1 in e2
 * e --> v1
 * [v1/x] e2 --> v2
 
 * let x = 
     let y = 2 in y+2 in
   foo(x)
     
 *)

let get_fresh_var = 
  let i (*addr*) = ref 0 in
  fun () ->
    begin
      i(*addr*) := !i(*!addr*) + 1;
      "v_"^(string_of_int !i)
    end
(*
 * get_fresh_var = 
 *  fun () ->
 *   begin
 *     addr := !addr + 1;
       "v_"^(string_of_int !addr)
 *   enb
 *)

val get_fresh_var : unit -> string = <fun>


In [29]:
get_fresh_var ()

- : string = "v_1"


In [30]:
let foo = 
 let x = 2 in
 fun () -> x*2
 
(*
 * foo = fun () -> 2*2
\*)

val foo : unit -> int = <fun>


In [31]:
foo ()

- : int = 4


In [32]:
get_fresh_var() 
(*
 * actual type: get_fresh_var: unit -> string
 * hypothetical type: get_fresh_var: mem -> (string * mem) 
 * get_fresh_var is a side-effecting function
 * side effects are Bad!!
 *)

- : string = "v_2"


In [33]:
let count = ref 0

val count : int ref = {contents = 0}


In [34]:
!count

- : int = 0


In [35]:

(*
 * Real type : int -> int
 * Hypothetical type of fib:
 *  int -> mem -> (mem*int)
 *)
let rec fib (n:int) = 
  begin
  count := !count + 1;
  if n <= 2 then 1 
  else  fib (n-1) + fib (n-2)
  end

val fib : int -> int = <fun>


In [36]:

fib 3

- : int = 2


In [37]:
!count

- : int = 3


In [38]:
count := 0

- : unit = ()


In [39]:

fib 4

- : int = 3


In [40]:
!count

- : int = 5


In [41]:
fib 3

- : int = 2


In [42]:
!count

- : int = 8


In [43]:
type state = int
(*
 * "threading the state"
 *)
let rec fib_aux (n:int) (s:state) : (state*int) = 
  if n<=2 then (s+1,1)
  else
    let (s1,v1) = fib_aux (n-1) s in
    let (s2,v2) = fib_aux (n-2) s1 in
    (s2+1, v1+v2) 

type state = int


val fib_aux : int -> state -> state * int = <fun>


In [44]:
let fib n = fib_aux n 0

val fib : int -> state * int = <fun>


In [45]:
fib_aux

- : int -> state -> state * int = <fun>


In [46]:
type stateful_computation_returning_int = state -> state*int

type stateful_computation_returning_int = state -> state * int


In [47]:
let fib_aux: (int -> stateful_computation_returning_int) = fib_aux

val fib_aux : int -> stateful_computation_returning_int = <fun>


In [71]:
(*
 * A computation on integer state
 *)
module IntStateComp = struct
  type state = int
  (*
   * This is a stateful compuation on integer state that returns a value of type 'a
   *)
  type 'a t = state (* initial state *) -> state(* final state *) * 'a (* return value *)
  
  let bind (c1: 'a t) (c2: 'a -> 'b t) : 'b t =
    fun (s:state) -> 
      let (s1, x) = c1 s in
      c2 x s1
      
   let return (x:'a) : 'a t = fun (s:state) -> (s,x)
   
   (* let inc : unit t = fun (s:state) -> (s+1,()) *)
   
   let get: state t = fun (s:state) -> (s,s)
   
   let put (new_state:state) : unit t = fun (old_state:state) -> (new_state,())
   
   let run_state (state_comp: 'a t) (init_state:state) : (state*'a) = state_comp init_state
end

module IntStateComp :
  sig
    type state = int
    type 'a t = state -> state * 'a
    val bind : 'a t -> ('a -> 'b t) -> 'b t
    val return : 'a -> 'a t
    val get : state t
    val put : state -> unit t
    val run_state : 'a t -> state -> state * 'a
  end


In [59]:
(* 
 * bool IntStateComp.t
 *)
let state_is_even = fun (s:state) : (state*bool) -> (s, s mod 2 = 0)

val state_is_even : IntStateComp/2.state -> IntStateComp/2.state * bool =
  <fun>


In [60]:
(*
 * bool -> unit IntStateComp.t
 *)
let inc_if_true (b:bool) (s:state) : (state*unit) = 
   if b then (s+1,()) else (s,())

val inc_if_true : bool -> IntStateComp/2.state -> IntStateComp/2.state * unit =
  <fun>


In [61]:
let inc_if_state_is_even (s:state) : (state*unit) = 
  let (s1,b) = state_is_even s in
  inc_if_true b s1

val inc_if_state_is_even :
  IntStateComp/2.state -> IntStateComp/2.state * unit = <fun>


In [62]:
let (>>=) = IntStateComp.bind

val ( >>= ) :
  'a IntStateComp.t -> ('a -> 'b IntStateComp.t) -> 'b IntStateComp.t = <fun>


In [63]:
let (let*) = IntStateComp.bind

val ( let* ) :
  'a IntStateComp.t -> ('a -> 'b IntStateComp.t) -> 'b IntStateComp.t = <fun>


In [64]:
let inc = IntStateComp.inc

val inc : unit IntStateComp.t = <fun>


In [89]:
open IntStateComp
(*
 * Monad is essentially a programmable semi-colon. 
 * Haskell syntax makes this explicit.
 *
  do {
      old_state <- get;
      let new_state = old_state + 1 in
      put new_state;
      if (n<=2) then
        return 1
      else 
        v1 <- fib_aux (n-1);
        v2 <- fub_aux (n-2);
        n <- get;
        Printf.printf ...;
        return (v1+v2);
      }
  }
*)
let rec fib_aux (n:int) : int IntStateComp.t = 
  let* old_state = get in
  let new_state = old_state+1 in
  let* () = put new_state in
  if n<=2 then return 1
  else
    let* v1 = fib_aux (n-1) in
    let* v2 = fib_aux (n-2) in
    let* n = get in
    let _ = Printf.printf "So far %d recursive calls\n" n in
    return (v1+v2)

val fib_aux : int -> int IntStateComp.t = <fun>


In [85]:
let rec fib (n:int) = 
  let _ = count := !count + 1 in
  if n <= 2 then 1 
  else
   let v1 = fib (n-1) in
   let v2 = fib (n-2) in

   v1+v2

val fib : int -> int = <fun>


In [86]:
let fib_3_comp = (fib_aux 3)

val fib_3_comp : int IntStateComp.t = <fun>


In [87]:
IntStateComp.run_state fib_3_comp 0 

- : IntStateComp.state * int = (3, 2)


In [88]:
flush stdout

So far 3 recursive calls


- : unit = ()


In [90]:
module type STATE = sig
  type state
  type 'a t
  val return : 'a -> 'a t
  val bind: 'a t -> ('a -> 'b t) -> 'b t
  val get: state t
  val put: state -> unit t
  val run_state: state -> 'a t -> state*'a
end

module type STATE =
  sig
    type state
    type 'a t
    val return : 'a -> 'a t
    val bind : 'a t -> ('a -> 'b t) -> 'b t
    val get : state t
    val put : state -> unit t
    val run_state : state -> 'a t -> state * 'a
  end


In [94]:
(*
module type STATE_ARG = struct
 type state
end
*)

module State(S: sig
                  type t
                end) : (STATE with type state = S.t) = struct
  type state = S.t
  (*
   * This is a stateful compuation on integer state that returns a value of type 'a
   *)
  type 'a t = state (* initial state *) -> state(* final state *) * 'a (* return value *)
  
  let bind (c1: 'a t) (c2: 'a -> 'b t) : 'b t =
    fun (s:state) -> 
      let (s1, x) = c1 s in
      c2 x s1
      
   let return (x:'a) : 'a t = fun (s:state) -> (s,x)
   
   let get: state t = fun (s:state) -> (s,s)
   
   let put (new_state:state) : unit t = fun (old_state:state) -> (new_state,())
   
   let run_state (init_state:state) (state_comp: 'a t)  : (state*'a) = state_comp init_state
end

module State :
  functor (S : sig type t end) ->
    sig
      type state = S.t
      type 'a t
      val return : 'a -> 'a t
      val bind : 'a t -> ('a -> 'b t) -> 'b t
      val get : state t
      val put : state -> unit t
      val run_state : state -> 'a t -> state * 'a
    end


(*
 * Stack machine
 * instructions:
 *   1. push
 *   2. add 
 *   3. ite 
 *)
```ocaml
module type NAIVE_STACK_MACHINE = sig
    type stack = int list
    val push: int -> stack -> stack
    val add: stack -> stack
    val ite: stack -> stack
end

(* threading the state *)

let init_stack = [] in
let s1 = push 2 init_stack in
let s2 = push 3 s1 in
let s2 = add s2 in
,...
```

In [95]:
module StackStateMonad = State(struct
                                type t = int list
                               end)

module StackStateMonad :
  sig
    type state = int list
    type 'a t
    val return : 'a -> 'a t
    val bind : 'a t -> ('a -> 'b t) -> 'b t
    val get : state t
    val put : state -> unit t
    val run_state : state -> 'a t -> state * 'a
  end


In [98]:
open StackStateMonad

  

In [100]:
let (let*) = bind

val ( let* ) :
  'a StackStateMonad.t ->
  ('a -> 'b StackStateMonad.t) -> 'b StackStateMonad.t = <fun>


In [101]:
let push (n:int) : unit t =
  let* s1 = get in
  put (n::s1)

val push : int -> unit StackStateMonad.t = <fun>


In [102]:
let add : unit t = 
  let* s1 = get in
  match s1 with
  | x::y::xs -> put ((x+y)::xs)
  | _ -> return ()

val add : unit StackStateMonad.t = <abstr>


In [103]:
let ite : unit t = 
  let* s1 = get in
  match s1 with
  | x::y::z::z'::xs when x=y -> put (z::xs)
  | x::y::z::z'::xs -> put (z'::xs)
  | _ -> return ()

val ite : unit StackStateMonad.t = <abstr>


In [106]:
let my_simple_stack_comp =  
   let* _ = push 2 in
   let* _ = push 3 in
   add

val my_simple_stack_comp : unit StackStateMonad.t = <abstr>


In [107]:
run_state [] my_simple_stack_comp

- : StackStateMonad.state * unit = ([5], ())


In [1]:
()

- : unit = ()


In [3]:
let prev_stack = (2,3)

val prev_stack : int * int = (2, 3)


In [4]:
let stack = ("hello", prev_stack)

val stack : string * (int * int) = ("hello", (2, 3))


In [6]:
(*
 * An algebraic data type for a simple pair
 *)

let mk_pair x y = (x,y) 

val mk_pair : 'a -> 'b -> 'a * 'b = <fun>


In [8]:
type ('a,'b) pair = 
  | Mkpair of 'a * 'b

type ('a, 'b) pair = Mkpair of 'a * 'b


In [10]:
type 'a my_list =
  | MyNil
  | MyCons of 'a * 'a list

type ('a,'b) pair_list = 
  | Nil
  | Cons of ('a*'b) * ('a,'b) pair_list

type 'a my_list = MyNil | MyCons of 'a * 'a list


type ('a, 'b) pair_list = Nil | Cons of ('a * 'b) * ('a, 'b) pair_list


In [11]:
type ('a,'b) tree = 
  | L
  | B of ('a,'b) tree * 'a * 'b * ('a,'b) tree 

type ('a, 'b) tree = L | B of ('a, 'b) tree * 'a * 'b * ('a, 'b) tree


In [12]:
module type STACK_OPS =
sig
  type ('s,'r,'a) t
  val add : unit -> (int * (int * 's), 
                     int * 's, 
                     unit) t
  val _if_ : unit -> (bool * ('a * ('a * 's)), 
                      'a * 's, 
                      unit) t
  val push_const : 'a -> ('s, 
                          'a * 's, 
                          unit) t
end

module type STACK_OPS =
  sig
    type ('s, 'r, 'a) t
    val add : unit -> (int * (int * 's), int * 's, unit) t
    val _if_ : unit -> (bool * ('a * ('a * 's)), 'a * 's, unit) t
    val push_const : 'a -> ('s, 'a * 's, unit) t
  end


In [13]:
(*
 * This is a computation that operates on initial state of type 's
 * returns a new state of type 'r
 * and also a return value of type 'a
 *)
type ('s,'r,'a) t = 's -> ('r*'a)

type ('s, 'r, 'a) t = 's -> 'r * 'a


In [15]:
let return (x:'a): ('s,'s,'a) t = fun (init_state:'s) -> (init_state,x)

val return : 'a -> ('s, 's, 'a) t = <fun>


In [17]:
let bind (c1: ('s,'r,'a) t) (c2: 'a -> ('r,'u,'b) t) : ('s,'u,'b) t = 
  fun (s) -> 
      let (s1, x) = c1 s in
      c2 x s1

val bind : ('s, 'r, 'a) t -> ('a -> ('r, 'u, 'b) t) -> ('s, 'u, 'b) t = <fun>
